# 🚀 Hybrid QML vs Classical SOTA — Full Training & Comparison

This notebook lets you train **either or both** of:
1. **Hybrid QML** — ResNet18 (frozen) + 10-qubit PQC Ensemble (~92–93% acc)
2. **Classical SOTA** — ResNet50 two-stage fine-tune baseline (~strong reference)

Then run a **side-by-side evaluation** (accuracy, F1, AUC, precision, recall).

> ⚠️ **Required:** GPU runtime — go to **Runtime → Change runtime type → T4 GPU**

In [1]:
# ══════════════════════════════════════════════════
# 🔧 USER CONFIGURATION — Edit these as needed
# ══════════════════════════════════════════════════

# Which models to train? Set True/False independently.
TRAIN_HYBRID    = True   # Hybrid QML (ResNet18 + Quantum Ensemble)
TRAIN_CLASSICAL = True   # Classical SOTA (ResNet50, two-stage fine-tune)

# Run head-to-head comparison after training?
RUN_COMPARISON  = True

# Dataset path on Google Drive
DRIVE_DATASET_PATH = '/content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset'

# Where to persist outputs on Drive
DRIVE_OUTPUT_PATH  = '/content/drive/MyDrive/qml_medical_imaging_outputs'

# Repo
REPO_URL    = 'https://github.com/DreamX55/QML-for-Medical-Imaging.git'
PROJECT_KEY = 'qml_medical_imaging'
PROJECT_ROOT = f'/content/{PROJECT_KEY}'

# Training hyper-params
BATCH_SIZE         = 32
HYBRID_EPOCHS      = 15          # Hybrid model epochs (frozen backbone)
CLASSICAL_STAGE1   = 10          # Classical: Stage 1 (frozen backbone)
CLASSICAL_STAGE2   = 15          # Classical: Stage 2 (fine-tune layer4)
NUM_WORKERS        = 2
DEVICE             = 'cpu'      # 'cuda' | 'cpu'

# Output sub-dirs (relative to DRIVE_OUTPUT_PATH)
HYBRID_OUT_DIR     = 'outputs/high_perf_resnet'
CLASSICAL_OUT_DIR  = 'outputs/classical_sota'
COMPARISON_JSON    = 'outputs/comparison_results.json'

print('✅ Configuration set.')

✅ Configuration set.


In [2]:
# ══════════════════════════════════════════════════
# 1. Mount Drive & Clone / Update Repo
# ══════════════════════════════════════════════════
import os, shutil
from google.colab import drive
drive.mount('/content/drive')

if os.path.exists(PROJECT_ROOT):
    print(f'Repo exists. Pulling latest...')
    %cd {PROJECT_ROOT}
    !git pull
else:
    print(f'Cloning repo...')
    %cd /content
    !git clone {REPO_URL} {PROJECT_KEY}

%cd {PROJECT_ROOT}
!pip install -q -r requirements.txt
!pip install -q pennylane-lightning
print('✅ Environment ready.')

Mounted at /content/drive
Cloning repo...
/content
Cloning into 'qml_medical_imaging'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 125 (delta 50), reused 103 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (125/125), 124.02 KiB | 7.29 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/qml_medical_imaging
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 22.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 131.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.6/935.6 kB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8

In [3]:
# ══════════════════════════════════════════════════
# 2. Check Hardware (GPU or CPU)
# ══════════════════════════════════════════════════
import torch
print(f'PyTorch : {torch.__version__}')
print(f'CUDA    : {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'GPU     : {torch.cuda.get_device_name(0)}')
    import subprocess
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print(result.stdout)
else:
    print('ℹ️  Running on CPU — training will be slower but fully functional.')
    print('   Tip: Quantum simulation on CPU is ~2–5 min/epoch for small batches.')


PyTorch : 2.10.0+cu128
CUDA    : True
GPU     : Tesla T4
Sat Feb 21 07:07:53 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   35C    P8             12W /   70W |       3MiB /  15360MiB |      0%      Default |
|                                         |                        |               

In [4]:
# ══════════════════════════════════════════════════
# 3. Link Data & Outputs from Drive
# ══════════════════════════════════════════════════
def force_symlink(src, dst):
    if os.path.exists(dst) or os.path.islink(dst):
        if os.path.isdir(dst) and not os.path.islink(dst):
            shutil.rmtree(dst)
        else:
            os.remove(dst)
    os.symlink(src, dst)
    print(f'Linked  {src}  →  {dst}')

# Data
data_link = os.path.join(PROJECT_ROOT, 'data/brain_mri')
os.makedirs(os.path.join(PROJECT_ROOT, 'data'), exist_ok=True)
force_symlink(DRIVE_DATASET_PATH, data_link)

# Outputs
if not os.path.exists(DRIVE_OUTPUT_PATH):
    os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)
output_link = os.path.join(PROJECT_ROOT, 'outputs')
force_symlink(DRIVE_OUTPUT_PATH, output_link)

print('✅ Data and outputs linked to Drive.')

Linked  /content/drive/MyDrive/medical_imaging_data/brain_tumor_dataset  →  /content/qml_medical_imaging/data/brain_mri
Linked  /content/drive/MyDrive/qml_medical_imaging_outputs  →  /content/qml_medical_imaging/outputs
✅ Data and outputs linked to Drive.


---
## 🧪 Training
The next two cells train the **Hybrid QML** and **Classical SOTA** models respectively.
Each cell only runs if the corresponding flag is set to `True` above.

In [1]:
# ══════════════════════════════════════════════════
# 4a. Train Hybrid QML Model (ResNet18 + Quantum Ensemble)
# ══════════════════════════════════════════════════
if TRAIN_HYBRID:
    print('🔬 Training Hybrid QML model...')
    !python main.py --mode train \
        --data_dir data/brain_mri \
        --project_name qml_high_perf \
        --model_type hybrid \
        --backbone resnet18 \
        --freeze_backbone \
        --epochs {HYBRID_EPOCHS} \
        --batch_size {BATCH_SIZE} \
        --num_workers {NUM_WORKERS} \
        --quantum_device default.qubit \
        --device {DEVICE} \
        --output_dir {HYBRID_OUT_DIR}
else:
    print('⏭️  Skipping Hybrid QML training (TRAIN_HYBRID=False)')

NameError: name 'TRAIN_HYBRID' is not defined

In [6]:
# ══════════════════════════════════════════════════
# 4b. Train Classical SOTA Model (ResNet50, Two-Stage)
# ══════════════════════════════════════════════════
if TRAIN_CLASSICAL:
    print('🏋️  Training Classical SOTA model (ResNet50)...')
    !python training/train_classical_sota.py \
        --data_dir data/brain_mri \
        --output_dir {CLASSICAL_OUT_DIR} \
        --epochs_stage1 {CLASSICAL_STAGE1} \
        --epochs_stage2 {CLASSICAL_STAGE2} \
        --batch_size {BATCH_SIZE} \
        --num_workers {NUM_WORKERS} \
        --device {DEVICE}
else:
    print('⏭️  Skipping Classical SOTA training (TRAIN_CLASSICAL=False)')

🏋️  Training Classical SOTA model (ResNet50)...
^C


---
## 📊 Head-to-Head Evaluation
Loads both saved checkpoints and evaluates them on the **same test set**.
Results are printed side-by-side and saved to Drive.

In [7]:
# ══════════════════════════════════════════════════
# 5. Side-by-Side: Hybrid QML vs Classical SOTA
# ══════════════════════════════════════════════════
HYBRID_CKPT    = f'{HYBRID_OUT_DIR}/checkpoints/best_model.pt'
CLASSICAL_CKPT = f'{CLASSICAL_OUT_DIR}/best_model.pt'

if RUN_COMPARISON:
    # Verify both checkpoints exist before running
    import os
    missing = [p for p in [HYBRID_CKPT, CLASSICAL_CKPT] if not os.path.exists(p)]
    if missing:
        print('⚠️  Missing checkpoints — train the models first:')
        for m in missing:
            print(f'   {m}')
    else:
        print('⚖️  Running head-to-head comparison...')
        !python evaluation/evaluate_classical_vs_hybrid.py \
            --data_dir data/brain_mri \
            --hybrid_checkpoint {HYBRID_CKPT} \
            --classical_checkpoint {CLASSICAL_CKPT} \
            --batch_size {BATCH_SIZE} \
            --num_workers {NUM_WORKERS} \
            --device {DEVICE} \
            --output_json {COMPARISON_JSON}
        print(f'\n✅ Comparison JSON saved → {COMPARISON_JSON}')
else:
    print('⏭️  Skipping comparison (RUN_COMPARISON=False)')

⚠️  Missing checkpoints — train the models first:
   outputs/high_perf_resnet/checkpoints/best_model.pt
   outputs/classical_sota/best_model.pt


In [8]:
# ══════════════════════════════════════════════════
# 6. Visualise Results (reads comparison JSON)
# ══════════════════════════════════════════════════
import json, os
import matplotlib.pyplot as plt
import numpy as np

if not os.path.exists(COMPARISON_JSON):
    print(f'No comparison JSON found at {COMPARISON_JSON}. Run Cell 5 first.')
else:
    with open(COMPARISON_JSON) as f:
        res = json.load(f)

    metrics_ = ['accuracy', 'f1_macro', 'roc_auc', 'precision_macro', 'recall_macro']
    labels_  = ['Accuracy', 'F1 (macro)', 'ROC-AUC', 'Precision', 'Recall']

    h_vals = [res['hybrid'][m] for m in metrics_]
    c_vals = [res['classical'][m] for m in metrics_]

    x = np.arange(len(labels_))
    w = 0.35

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.bar(x - w/2, h_vals, w, label='Hybrid QML (ResNet18+Q)', color='steelblue', alpha=0.85)
    ax.bar(x + w/2, c_vals, w, label='Classical SOTA (ResNet50)', color='tomato', alpha=0.85)

    ax.set_ylabel('Score', fontsize=12)
    ax.set_title('Hybrid QML vs Classical SOTA — Test Set Metrics', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(labels_, fontsize=11)
    ax.set_ylim(0, 1.05)
    ax.legend(fontsize=11)
    ax.yaxis.grid(True, alpha=0.4)

    for i, (hv, cv) in enumerate(zip(h_vals, c_vals)):
        ax.text(i - w/2, hv + 0.01, f'{hv:.3f}', ha='center', va='bottom', fontsize=9, color='steelblue')
        ax.text(i + w/2, cv + 0.01, f'{cv:.3f}', ha='center', va='bottom', fontsize=9, color='tomato')

    plt.tight_layout()
    plot_path = COMPARISON_JSON.replace('.json', '_bar.png')
    plt.savefig(plot_path, dpi=150)
    plt.show()
    print(f'✅ Plot saved → {plot_path}')

No comparison JSON found at outputs/comparison_results.json. Run Cell 5 first.
